In [1]:
import os
from datetime import datetime
import time
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import numpy as np
import cv2
import easyocr
reader = easyocr.Reader(['en'])



CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [2]:
Num_model = torch.hub.load(r'ultralytics/yolov5', 'custom', path=r'C:\Users\akhas\Desktop\Project\Num_net.pt', force_reload=True)
Vehicle_model = torch.hub.load(r'ultralytics/yolov5', 'custom', path=r'C:\Users\akhas\Desktop\Project\Vehicle_NET.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\akhas/.cache\torch\hub\master.zip
YOLOv5  2022-12-28 Python-3.10.0 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\akhas/.cache\torch\hub\master.zip
YOLOv5  2022-12-28 Python-3.10.0 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7026307 parameters, 0 gradients
Adding AutoShape... 


In [6]:
def read_num(model,frame):
  classes = model.names
  def class_to_label(x):     
       return classes[int(x)]   
  results = model(frame)
  labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
  n = len(labels)
  x_shape, y_shape = frame.shape[1], frame.shape[0]
  for i in range(n):
        row = cord[i]
        if row[4] > 0.8:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (225, 0, 225)
                              
                crop = frame[y1:y2, x1:x2]                
                gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
                gray = cv2.resize( gray, None, fx = 7, fy = 7, interpolation = cv2.INTER_CUBIC)  
                ret, thresh1 = cv2.threshold(gray, 90, 250, cv2.THRESH_BINARY)
                results = reader.readtext(thresh1)
                print(results)
                for (bbox, text, prob) in results:
                   print("[INFO] {:.4f}: {}".format(prob, text)) 
                return [text,prob]
                
                #cv2.putText(frame,class_to_label(results.xyxyn[0][:, -1][i]),(x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 2, bgr, 4)
    
    

In [9]:
def process_image(model,frame):
  classes = model.names
  def class_to_label(x):     
       return classes[int(x)]  
  results = model(frame)
  labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
  n = len(labels)
  x_shape, y_shape = frame.shape[1], frame.shape[0]
  for i in range(n):
        row = cord[i]
        if row[4] > 0.5:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (225, 0, 225)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 4)  
                
                crop = frame[y1:y2, x1:x2]

                num = read_num(Num_model,crop)

                lab = class_to_label(results.xyxyn[0][:, -1][i])
                #string = lab+"        "+num[0]+num[1]
                #print(lab)
                #print(num)
               

                cv2.putText(frame,lab,(x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 2, bgr, 4)
                resized_img = cv2.resize(frame,(480,720))
                

                cv2.imshow('image',resized_img)
                cv2.waitKey(1)
                
                
               # if cv2.waitKey(1) == 2:
                 #  break 

In [10]:
cap = cv2.VideoCapture('video.mp4')
while(cap.isOpened()):
  ret, frame = cap.read()  
  if ret == True:
    process_image(Vehicle_model,frame)
  


[([[117, 34], [512, 34], [512, 127], [117, 127]], '~,C :<', 0.14622928181815506)]
[INFO] 0.1462: ~,C :<
[([[130, 38], [535, 38], [535, 134], [130, 134]], '1, C :82,', 0.13334679490638032)]
[INFO] 0.1333: 1, C :82,
[([[123, 35], [538, 35], [538, 137], [123, 137]], '~C :85', 0.10683442570927956)]
[INFO] 0.1068: ~C :85
[([[122, 40], [537, 40], [537, 140], [122, 140]], 'LG :589', 0.17904363055870504)]
[INFO] 0.1790: LG :589
[([[124, 40], [536, 40], [536, 140], [124, 140]], 'L6 :55)', 0.09925736973860748)]
[INFO] 0.0993: L6 :55)
[([[183, 40], [544, 40], [544, 141], [183, 141]], '76:58', 0.23705700278452724)]
[INFO] 0.2371: 76:58
[([[100, 46], [545, 46], [545, 145], [100, 145]], 'N :: 5 :589', 0.14543453620287913)]
[INFO] 0.1454: N :: 5 :589
[([[70, 43], [543, 43], [543, 145], [70, 145]], '{n :! G :5S,', 0.0615242536763107)]
[INFO] 0.0615: {n :! G :5S,
[([[64, 47], [557, 47], [557, 156], [64, 156]], '{NJ: G :55,', 0.10381025566104533)]
[INFO] 0.1038: {NJ: G :55,
[([[66, 39], [565, 39], [565,

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!wdpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!wapt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!wapt-get update
!wapt-get install cuda